In [1]:
import subprocess
import os

import pandas as pd

import requests
from bs4 import BeautifulSoup

import json


pd.set_option('display.max_rows', 500)

![CRISP_DM](../reports/figures/CRISP_DM.png)

# Data Understanding

* RKI, webscrape (webscraping) https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html
* John Hopkins (GITHUB) https://github.com/CSSEGISandData/COVID-19.git
* REST API services to retreive data https://npgeo-corona-npgeo-de.hub.arcgis.com/

## GITHUB csv data

git clone/pull https://github.com/CSSEGISandData/COVID-19.git

In [2]:

git_pull = subprocess.Popen( "git pull" , 
                     cwd = os.path.dirname( '../data/raw/COVID-19/' ), 
                     shell = True, 
                     stdout = subprocess.PIPE, 
                     stderr = subprocess.PIPE )
(out, error) = git_pull.communicate()


print("Error : " + str(error)) 
print("out : " + str(out))

Error : b''
out : b'Already up to date.\n'


In [3]:
data_path='../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
pd_raw=pd.read_csv(data_path)

In [4]:
pd_raw.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/29/20,7/30/20,7/31/20,8/1/20,8/2/20,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,36471,36542,36675,36710,36710,36747,36782,36829,36896,37015
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,5105,5197,5276,5396,5519,5620,5750,5889,6016,6151
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,29229,29831,30394,30950,31465,31972,32504,33055,33626,34155
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,918,922,925,925,925,937,939,939,944,955
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,1078,1109,1148,1164,1199,1280,1344,1395,1483,1538


## Webscrapping

In [5]:
page = requests.get("https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html")

In [6]:
soup = BeautifulSoup(page.content, 'html.parser')

In [7]:
html_table=soup.find('table') # find the table, attention this works if one table exists

In [8]:
all_rows=html_table.find_all('tr')

In [9]:
final_data_list=[]

In [10]:
for pos,rows in enumerate(all_rows):
   
    col_list=[each_col.get_text(strip=True) for each_col in rows.find_all('td')] #td for data element
    final_data_list.append(col_list)
    

In [11]:
pd_daily_status=pd.DataFrame(final_data_list).dropna().rename(columns={0:'state',
                                                       1:'cases',
                                                       2:'changes',
                                                       3:'cases_per_100k',
                                                       4:'fatal',
                                                       5:'comment'})

In [12]:
pd_daily_status.head()

,state,cases,changes,cases_per_100k,fatal,comment
2,Baden-Württem­berg,37.749,+107,341,"3,1",1.858
3,Bayern,51.794,+84,696,"5,3",2.626
4,Berlin,9.640,+97,336,"9,0",224
5,Branden­burg,3.622,+22,48,"1,9",169
6,Bremen,1.800,+2,19,"2,8",56


## REST API calls

In [13]:
## data request for Germany
data=requests.get('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/Coronaf%C3%A4lle_in_den_Bundesl%C3%A4ndern/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')

In [14]:
json_object=json.loads(data.content) 

In [15]:
type(json_object)

dict

In [16]:
json_object.keys()

dict_keys(['objectIdFieldName', 'uniqueIdField', 'globalIdFieldName', 'geometryProperties', 'geometryType', 'spatialReference', 'fields', 'features'])

In [17]:
full_list=[]
for pos,each_dict in enumerate (json_object['features'][:]):
    full_list.append(each_dict['attributes'])
    

In [18]:
pd_full_list=pd.DataFrame(full_list)
pd_full_list.head()

,OBJECTID_1,LAN_ew_AGS,LAN_ew_GEN,LAN_ew_BEZ,LAN_ew_EWZ,OBJECTID,Fallzahl,Aktualisierung,AGS_TXT,GlobalID,faelle_100000_EW,Shape__Area,Shape__Length,Death
0,1,01,Schleswig-Holstein,Land,2896712,15,3578,1596837600000,01,fc5ba936-c95c-432c-8a33-9eb2f30b660f,123.519356,4.573731e+10,2.881496e+06,158
1,2,02,Hamburg,Freie und Hansestadt,1841179,6,5610,1596837600000,02,0f3e860c-5181-4d3f-a421-1d51f50315ea,304.696067,2.089396e+09,4.188002e+05,262
2,3,03,Niedersachsen,Land,7982448,9,14911,1596837600000,03,3fd77024-c29b-4843-9be8-682ad48e60c9,186.797333,1.299836e+11,4.008988e+06,654
3,4,04,Bremen,Freie Hansestadt,682986,5,1800,1596837600000,04,4132268b-54de-4327-ac1e-760e915112f1,263.548594,1.119157e+09,3.357177e+05,56
4,5,05,Nordrhein-Westfalen,Land,17932651,10,51330,1596837600000,05,561d658f-3ee5-46e3-bc95-3528c6558ab9,286.237657,8.782936e+10,2.648673e+06,1759


In [19]:
pd_full_list.to_csv('../data/raw/NPGEO/GER_state_data.csv',sep=';')

In [20]:
pd_full_list.shape[0]

16

# API access via REST service, e.g. USA data 

example of a REST conform interface (attention registration mandatory)

www.smartable.ai

In [21]:
# US for full list
headers = {
    'Cache-Control': 'no-cache',
    'Subscription-Key': '28ee4219700f48718be78b057beb7eb4',
}

response = requests.get('https://api.smartable.ai/coronavirus/stats/US', headers=headers)
print(response)

<Response [200]>


In [22]:
US_dict=json.loads(response.content) # imports string
with open('../data/raw/SMARTABLE/US_data.json', 'w') as outfile:
    json.dump(US_dict, outfile,indent=2)

In [23]:
print(json.dumps(US_dict,indent=2)) #string dump

{
  "location": {
    "long": -95.712891,
    "countryOrRegion": "United States",
    "provinceOrState": null,
    "county": null,
    "isoCode": "US",
    "lat": 37.09024
  },
  "updatedDateTime": "2020-08-08T10:02:34.5589018Z",
  "stats": {
    "totalConfirmedCases": 5018201,
    "newlyConfirmedCases": 69,
    "totalDeaths": 171189,
    "newDeaths": 7,
    "totalRecoveredCases": 2546534,
    "newlyRecoveredCases": 491,
    "history": [
      {
        "date": "2020-01-22T00:00:00",
        "confirmed": 1,
        "deaths": 0,
        "recovered": 0
      },
      {
        "date": "2020-01-23T00:00:00",
        "confirmed": 1,
        "deaths": 0,
        "recovered": 0
      },
      {
        "date": "2020-01-24T00:00:00",
        "confirmed": 2,
        "deaths": 0,
        "recovered": 0
      },
      {
        "date": "2020-01-25T00:00:00",
        "confirmed": 2,
        "deaths": 0,
        "recovered": 0
      },
      {
        "date": "2020-01-26T00:00:00",
        "confir

# Individual States US

In [24]:
US_dict['stats']['breakdowns'][0]

{'location': {'long': 144.793731,
  'countryOrRegion': 'United States',
  'provinceOrState': 'Guam',
  'county': None,
  'isoCode': None,
  'lat': 13.444304},
 'totalConfirmedCases': 32,
 'newlyConfirmedCases': 0,
 'totalDeaths': 1,
 'newDeaths': 0,
 'totalRecoveredCases': 0,
 'newlyRecoveredCases': 0}

In [25]:
full_list_US_country=[]
for pos,each_dict in enumerate (US_dict['stats']['breakdowns'][:]):
    flatten_dict=each_dict['location']
    flatten_dict.update(dict(list(US_dict['stats']['breakdowns'][pos].items())[1: 7]) 
    )
    full_list_US_country.append(flatten_dict)

In [26]:
pd.DataFrame(full_list_US_country).to_csv('../data/raw/SMARTABLE/full_list_US_country.csv',sep=';',index=False)